## BART
BART模型是Facebook AI推出的一个有趣的模型。它基于Transformer架构，本质上是一个降噪自编码器，是通过重建受损文本进行训练的。

BART模型的**编码器是双向**的，这意味着它可以从两个方向（从左到右和从右到左）读取一个句子.

但BART模型的**解码器是单向**的，它只能从左到右读取一个句子。

因此，在BART模型中，我们有一个双向编码器（针对两个方向）和一个自回归解码器（针对单一方向）。

![](4.png)

### 受损文本的构造
- 标记掩盖
- 标记删除
- 标记填充
- 句子重排
- 文档轮换

### BART的使用

In [1]:
! pip install Transformers==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 23.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


首先，从Transformers库中导入用于分词的BartTokenizer和用于文本摘要任务的BartForConditionalGeneration。

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [3]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


设置原始文本以获得文本摘要。

In [4]:
text = """Machine learning (ML) is the study of computer algorithms that improve automatically through experience.
It is seen as a subset of artificial intelligence.
Machine learning algorithms build a mathematical model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to do so.
Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop conventional algorithms to perform the needed tasks."""

对文本进行标记

In [6]:
inputs = tokenizer([text], max_length=1024, return_tensors='pt')
print(inputs)

{'input_ids': tensor([[    0, 46100,  2239,    36, 10537,    43,    16,     5,   892,     9,
          3034, 16964,    14,  1477,  6885,   149,   676,     4, 50118,   243,
            16,   450,    25,    10, 37105,     9,  7350,  2316,     4,  1437,
         50118, 46100,  2239, 16964,  1119,    10, 30412,  1421,   716,    15,
          7728,   414,     6,   684,    25,  1058,   414,     6,    11,   645,
             7,   146, 12535,    50,  2390,   396,   145, 16369, 30825,     7,
           109,    98,     4, 50118, 46100,  2239, 16964,    32,   341,    11,
            10,  1810,  3143,     9,  2975,     6,   215,    25,  1047, 35060,
             8,  3034,  3360,     6,   147,    24,    16,  1202,    50,  4047,
         29358,  4748,     7,  2179,  9164, 16964,     7,  3008,     5,   956,
          8558,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

　获取摘要ID，也就是模型生成的标记ID。

In [7]:
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)

现在，对摘要ID进行解码，得到相应的标记（单词）。

In [8]:
summary = ([tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=False) for i in summary_ids])

In [9]:
print(summary)

['Machine learning is the study of computer algorithms that improve automatically through experience. It is seen as a subset of artificial intelligence. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop conventional algorithms to perform the needed tasks.']
